In [1]:
# verify_boss.py
import requests
import time
import random
import pprint

SEARCH_PAGE = "https://www.zhipin.com/web/geek/job"  # 预热页
API_URL = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"

def cookie_str_to_dict(cookie_str: str) -> dict:
    jar = {}
    for kv in cookie_str.split(';'):
        kv = kv.strip()
        if not kv or '=' not in kv:
            continue
        k, v = kv.split('=', 1)
        jar[k.strip()] = v.strip()
    return jar

def verify_once(raw_cookie: str, query="Python高级开发工程师", city="100010000"):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
        "Referer": "https://www.zhipin.com/",
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept-Language": "zh-CN,zh;q=0.9",
    }

    params = {
        "scene": "1",
        "query": query,
        "city": city,
        "experience": "106",      # 可改
        "scale": "303,304,305",   # 可改
        "page": 1,
        "pageSize": 30
    }

    sess = requests.Session()
    sess.headers.update(headers)
    sess.cookies.update(cookie_str_to_dict(raw_cookie))

    # 1) 预热：先访问搜索页（GET），让服务端可能刷新 Cookie / 令牌
    try:
        r = sess.get(SEARCH_PAGE, params={"query": params["query"], "city": params["city"]}, timeout=12)
        r.raise_for_status()
        # 给服务器一点时间/节奏
        time.sleep(random.uniform(0.8, 1.3))
    except Exception as e:
        print("预热页访问失败：", e)
        # 仍继续尝试请求接口（有时候不需要预热也能成功）

    # 2) 真正 POST 请求（把参数放在 body）
    try:
        resp = sess.post(API_URL, data=params, timeout=12)
        resp.raise_for_status()
    except Exception as e:
        print("请求 joblist 接口失败：", e)
        return

    # 3) 解析并展示返回内容的关键信息
    text_snip = resp.text[:800]
    try:
        j = resp.json()
    except Exception as e:
        print("返回不是 JSON，可能被风控或登录失效。前800字符如下：")
        print(text_snip)
        return

    # 打印接口 code/message 以及 jobList 情况
    code = j.get("code")
    msg = j.get("message") or j.get("msg")
    print("接口返回 code:", code, " message:", msg)

    job_list = (j.get("zpData") or {}).get("jobList", [])
    print("jobList 长度:", len(job_list))

    if not job_list:
        print("注意：jobList 为空 → 可能 Cookie 无效 / 条件过窄 / 被风控。")
        # 打印整个响应的部分关键字段，便于排查
        pprint.pprint({k: j.get(k) for k in ("code", "message", "zpData")})
        return

    # 打印前 5 条简要信息
    print("\n示例职位（最多 5 条）：")
    for i, job in enumerate(job_list[:5], 1):
        print(f"--- #{i} ---")
        print("职位:", job.get("jobName"))
        print("公司:", job.get("brandName"))
        print("薪资:", job.get("salaryDesc"))
        print("地区:", job.get("cityName"))
        print("行业:", job.get("brandIndustry"))
        print("encryptJobId:", job.get("encryptJobId"))
        print()

if __name__ == "__main__":
    # --------- 把这里替换成你从浏览器复制的 Cookie 字符串（整串） ----------
    RAW_COOKIE = "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; wt2=D5HHFcwhyxdQ01sgtrmifW2zZ1rjjLskv6olsSpu45vNX08LSHmdAPpeS50jdaNEyYK8InyitQqt9sP68nx1Dow~~; wbg=0; zp_at=41c35eZXeQtAPNTeV030wsxJrGMPY2QsEsGV4gcKzSo~; __l=r=https%3A%2F%2Fwww.google.com%2F&l=%2Fnapi%2Fzpssrseo%2Fzhaopin%2F2d18dbe5285b5f040n172dq8%2F&s=1; __zp_seo_uuid__=60076b16-b36e-4ee7-a22d-3ffc5ea74c46; lastCity=100010000; __zp_stoken__=eeddfNjzDocK8wo3CuUEmEw4JCwdDJz08LW8%2FNihFN0M2PDc5RTY8Pxc3JhXCt0cmOWTDjDg8Kzw3RDw2Pjc3QRs8Q8S9wrc3PTDDjsK3RiY4X8OHCcKywrxZw7DCvQvDh8K8C8KTwrctwofCvCwqwonCvDc9Pzxjwrt2w4Fjwrh8w4M3PDw3PTcoOGFWYFg4N0pGWQpJXkdfZFULTlVVKT1ANjfCgsKmMjkSERMREggLCQsIFQ4UBwwMBw0HDAYNBw0GLzzCosK3wpvColQVw6bElsKdVMKfU8OswqfDhMKxxIJpw71ww7fCtMKkUFTDgcKKwrfCikjClsK8wr9MwrFecnRuXsK5f1NWaFVaV2RvblISZGVkXjgUWMKv; bst=V2R9MnF-L5219pVtRuyR8eKyO07DrWwCg~|R9MnF-L5219pVtRuyR8eKyO07DrVxiw~"

    # 可把 query 设成 "" 以广撒网测试
    verify_once(RAW_COOKIE, query="Python高级开发工程师", city="100010000")


接口返回 code: 0  message: Success
jobList 长度: 30

示例职位（最多 5 条）：
--- #1 ---
职位: 高级python工程师
公司: 灵匠科技
薪资: 25-50K·13薪
地区: 南京
行业: 互联网
encryptJobId: bdea2528354bab7f03J72N68EVtS

--- #2 ---
职位: 高级大数据开发工程师（python）
公司: 极联股份
薪资: 8-12K
地区: 哈尔滨
行业: 计算机软件
encryptJobId: 5d2b0feae95c710603Bz3t68FlBW

--- #3 ---
职位: 高级音视频开发工程师
公司: 之江实验室
薪资: 40-70K
地区: 珠海
行业: 学术/科研
encryptJobId: eb8061cbe68bc5e003xy2Nm_FFBY

--- #4 ---
职位: python高级开发工程师
公司: 北京海天起点技术...
薪资: 13-14K
地区: 株洲
行业: 计算机软件
encryptJobId: 508c31c6025f6dc103x93tS9GVFY

--- #5 ---
职位: python后端开发工程师
公司: 湘潭糖尿病医院
薪资: 10-15K
地区: 湘潭
行业: 医疗健康
encryptJobId: 27f5518d198b4f4903d_3d6-EFVS



In [2]:
import requests
import time
import random
import datetime
import re
import pandas as pd
from bs4 import BeautifulSoup

SEARCH_PAGE = "https://www.zhipin.com/web/geek/job"
API_URL = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"

# ====== 已更新的 Cookie ======
RAW_COOKIE = "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; wt2=D5HHFcwhyxdQ01sgtrmifW2zZ1rjjLskv6olsSpu45vNX08LSHmdAPpeS50jdaNEyYK8InyitQqt9sP68nx1Dow~~; wbg=0; zp_at=41c35eZXeQtAPNTeV030wsxJrGMPY2QsEsGV4gcKzSo~; __l=r=https%3A%2F%2Fwww.google.com%2F&l=%2Fnapi%2Fzpssrseo%2Fzhaopin%2F2d18dbe5285b5f040n172dq8%2F&s=1; __zp_seo_uuid__=60076b16-b36e-4ee7-a22d-3ffc5ea74c46; lastCity=100010000; __zp_stoken__=eeddfOTbDoMK8wofCtkMnExQGCQZDLTo2LCQ%2BOSpEN0E5NjY5Pzk2Phc9KQzCt8O6GDZew40bNiw2NkQ2OUQ2N0McNkLEvcK9ODcxOMK3w7wYOV%2FDjQbCsMK9WcOywroJw4bCvAnClMK9LMKHwrYrKDDCtz06RT1jwrl5w4Niwrh2w4Riwrl3w4M6PT08MzZjVmNZNj1KTVgMS15MXmJPC1VUUys9Qzc5woDCpjE4FBMTEhMGCQkICRMUFAwNCg0NDA0IBwcGBzE2wqTCvMO9w4bCqhXDrcSXwpvCsMKnXcOowrPEhUvCsFLDg3LCkV7CoWHCh07Ci3LCiMOBwrBfwrp%2FwpdjSFVRwoJZclJXZ8KCcHvCuEdwSBNiX2RlORLClnfDig%3D%3D; bst=V2R9MnF-L5219pVtRuyR8eKyO07DrTxyQ~|R9MnF-L5219pVtRuyR8eKyO07DrRxSU~"# 职位关键词（空=不限）
QUERY = ""

# 金融行业关键词正则
FINANCE_PAT = re.compile(
    r"(金融|银行|券商|证券|基金|保险|期货|信托|投行|资管|理财|财富|小贷|消金|保理|租赁|征信|清算|清结算|支付|互金|金融科技|FinTech)",
    re.IGNORECASE
)

# 多个 User-Agent 池（轮换使用）
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15"
]

def cookie_str_to_dict(cookie_str: str) -> dict:
    """将 Cookie 字符串转为字典"""
    jar = {}
    for kv in cookie_str.split(';'):
        kv = kv.strip()
        if not kv or '=' not in kv:
            continue
        k, v = kv.split('=', 1)
        jar[k.strip()] = v.strip()
    return jar

def looks_like_finance(brand_industry: str, brand_name: str) -> bool:
    """判断是否为金融相关（行业或公司名命中即可）"""
    t1 = brand_industry or ""
    t2 = brand_name or ""
    return bool(FINANCE_PAT.search(t1)) or bool(FINANCE_PAT.search(t2))

def get_random_headers() -> dict:
    """生成随机请求头"""
    return {
        "User-Agent": random.choice(USER_AGENTS),
        "Referer": "https://www.zhipin.com/web/geek/job",
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Origin": "https://www.zhipin.com",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"'
    }

def get_job_detail(sess: requests.Session, job_id: str) -> str:
    """获取职位详情描述"""
    if not job_id:
        return ""
    url = f"https://www.zhipin.com/job_detail/{job_id}.html"
    try:
        headers = get_random_headers()
        r = sess.get(url, headers=headers, timeout=15)
        if r.status_code != 200:
            return ""
        soup = BeautifulSoup(r.text, "html.parser")
        tag = soup.select_one(".job-sec-text") or soup.select_one(".job-detail") or soup.select_one(".detail-content")
        return tag.get_text(strip=True) if tag else ""
    except Exception as e:
        print(f"详情页获取失败 ({job_id}): {e}")
        return ""

def exponential_backoff_sleep(attempt: int, base_min=3.0, base_max=6.0):
    """指数退避延时策略"""
    multiplier = min(2 ** (attempt - 1), 8)  # 最大 8 倍
    min_sleep = base_min * multiplier
    max_sleep = base_max * multiplier
    sleep_time = random.uniform(min_sleep, max_sleep)
    print(f"💤 等待 {sleep_time:.1f} 秒（第 {attempt} 次请求）...")
    time.sleep(sleep_time)

def run(max_collect=50, page_size=15, max_pages_soft=50):
    """
    主采集函数（优化反爬策略）
    
    参数:
        max_collect: 单次运行最大采集数（建议不超过 50）
        page_size: 每页数量（降低到 15）
        max_pages_soft: 最大页数
    """
    sess = requests.Session()
    sess.cookies.update(cookie_str_to_dict(RAW_COOKIE))

    # 预热：模拟真实用户先访问搜索页
    print("🔥 正在预热连接...")
    try:
        headers = get_random_headers()
        sess.headers.update(headers)
        _ = sess.get(SEARCH_PAGE, params={"query": QUERY, "city": "100010000"}, timeout=15)
        time.sleep(random.uniform(3.0, 5.0))  # 预热延时增加到 3-5 秒
    except Exception as e:
        print(f"⚠️ 预热失败（可继续）: {e}")

    collected = []
    seen_ids = set()
    empty_hits = 0
    request_count = 0

    for page in range(1, max_pages_soft + 1):
        if len(collected) >= max_collect:
            print(f"✅ 已达目标数量 {max_collect}，停止采集")
            break

        form = {
            "scene": "1",
            "query": QUERY,
            "city": "100010000",
            "page": page,
            "pageSize": page_size
        }

        print(f"\n📄 正在抓取列表 第 {page} 页 ...")
        
        # 指数退避延时（第 2 页开始）
        if page > 1:
            exponential_backoff_sleep(page)

        try:
            # 每次请求更换 User-Agent
            headers = get_random_headers()
            sess.headers.update(headers)
            
            resp = sess.post(API_URL, data=form, timeout=15)
            resp.raise_for_status()
            j = resp.json()
            request_count += 1
        except requests.exceptions.HTTPError as e:
            print(f"❌ HTTP 错误: {e.response.status_code}")
            print(resp.text[:300])
            break
        except Exception as e:
            print(f"❌ 请求异常: {e}")
            if hasattr(resp, 'text'):
                print(resp.text[:300])
            break

        # 检查返回码
        code = j.get("code")
        msg = j.get("message") or j.get("msg") or ""
        
        if code == 37:
            print(f"❌ 触发风控 (code: 37): {msg}")
            print("\n💡 建议操作：")
            print("   1. 已采集数据将被保存")
            print("   2. 等待 30-60 分钟后重新运行")
            print("   3. 在浏览器中手动搜索职位并更新 Cookie")
            print(f"   4. 当前已请求 {request_count} 次，建议下次减少 max_collect")
            break
        elif code not in (0, "0", None):
            print(f"❌ 接口异常 code: {code}, message: {msg}")
            break

        # 解析职位列表
        job_list = (j.get("zpData") or {}).get("jobList", []) or []
        if not job_list:
            empty_hits += 1
            print(f"⚠️ 本页无数据（连续空页: {empty_hits}）")
            if empty_hits >= 2:  # 降低到 2 次就停止
                print("连续多页无数据，结束采集")
                break
            time.sleep(random.uniform(3.0, 5.0))
            continue
        
        empty_hits = 0
        print(f"✅ 本页获取到 {len(job_list)} 个职位")

        # 处理每个职位
        finance_count = 0
        for idx, job in enumerate(job_list, 1):
            if len(collected) >= max_collect:
                break

            job_id = job.get("encryptJobId")
            if not job_id or job_id in seen_ids:
                continue

            brand_industry = (job.get("brandIndustry") or "").strip()
            brand_name = (job.get("brandName") or "").strip()

            # 金融行业过滤
            if not looks_like_finance(brand_industry, brand_name):
                continue

            finance_count += 1
            print(f"  💼 [{finance_count}] {job.get('jobName')} - {brand_name}")

            # 获取详情（延时增加到 4-8 秒）
            desc = get_job_detail(sess, job_id)
            time.sleep(random.uniform(4.0, 8.0))

            item = {
                "职位": job.get("jobName"),
                "公司": brand_name,
                "薪资": job.get("salaryDesc"),
                "地区": job.get("cityName"),
                "经验": job.get("jobExperience"),
                "学历": job.get("jobDegree"),
                "公司规模": job.get("brandScaleName"),
                "行业": brand_industry,
                "福利标签": ",".join(job.get("welfareList", []) or []),
                "技能标签": ",".join(job.get("skills", []) or []),
                "职位描述": desc,
                "encryptJobId": job_id
            }

            collected.append(item)
            seen_ids.add(job_id)
            request_count += 1

        print(f"📊 本页符合条件: {finance_count} 个 | 已采集: {len(collected)}/{max_collect}")

    # 保存结果
    if not collected:
        print("\n❌ 没有采集到符合条件的职位。")
        return

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    fname = f"全国_金融行业_岗位数据_{ts}.xlsx"
    df = pd.DataFrame(collected)
    df.drop(columns=["encryptJobId"], errors="ignore", inplace=True)
    df.to_excel(fname, index=False)
    
    print(f"\n✅ 保存成功：{fname}")
    print(f"   📦 共 {len(df)} 条（去重后）")
    print(f"   🔢 总请求数: {request_count} 次")
    print(f"   📋 列: {', '.join(df.columns.tolist())}")

if __name__ == "__main__":
    # 首次建议只抓 20 条测试，验证通过后可改为 50
    run(max_collect=20, page_size=15)

🔥 正在预热连接...

📄 正在抓取列表 第 1 页 ...
❌ 触发风控 (code: 37): 您的访问行为异常.

💡 建议操作：
   1. 已采集数据将被保存
   2. 等待 30-60 分钟后重新运行
   3. 在浏览器中手动搜索职位并更新 Cookie
   4. 当前已请求 1 次，建议下次减少 max_collect

❌ 没有采集到符合条件的职位。


In [7]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime

class BossSpiderAPI:
    def __init__(self):
        """
           Cookie需要登录后去网页上获取
        """
        self.base_url = "https://www.zhipin.com/wapi/zpgeek/pc/recommend/job/list.json"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
            "Referer": "https://www.zhipin.com/",
            "Cookie": "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; lastCity=100010000; __zp_stoken__=5a4ffw48qw5YKPhRQC2QIF1N3SldZQGZqZlRLXW1WQFjCv2bCkMKiWsKvw4fCrsKRwrbCm8KqwoBmwqPCo8Ozw4TCnUzCrF7Cg8K3wq7CnMKTxJzDrMKFw4tgwpXCvcKmMjUCDQ0NAnF%2Bfn5xCRYDAwwUCwsLFA0CAgINPynDvcKgwpo9M0Y9IEtUVA9UWlpNWkAAUExOMj4CAAENPjc9PTIyw4XCoMKxw6%2FCusKgwrHDoMK6wq%2FCscOgPToyPcKyw7gvRkoBwrISDggBdw7CssOwAcONZS7CrTfCsEo2MzPCvcSzOTIfRz09RDIyOj0yLzIkw4JaIcKtKMK%2FwqQiPR1FMjI7Pz0yMjk9Oy4yRcKGIzI9LTkVAgoDCCxHwrHCgcKyw6cyMg%3D%3D"
        }
        self.detail_headers = {
            "User-Agent": self.headers["User-Agent"],
            "Cookie": self.headers["Cookie"]
        }
        self.params = {
    "scene": "1",
    "query": "金融",
    "city": "100000000",
    "page": 1,
    "pageSize": 30
        }

        self.data_list = []

    def fetch_data(self, max_pages=3):
        for page in range(1, max_pages + 1):
            print(f"抓取第 {page} 页...")
            self.params['page'] = page
            try:
                resp = requests.get(self.base_url, headers=self.headers, params=self.params)
                resp.raise_for_status()
            except requests.RequestException as e:
                print(f"请求失败：{e}")
                continue

            result = resp.json()
            job_list = result.get("zpData", {}).get("jobList", [])
            if not job_list:
                print("没有更多数据了")
                break

            for job in job_list:
                job_id = job.get("encryptJobId")
                job_desc = self.get_job_detail(job_id)

                item = {
                    "职位": job.get("jobName"),
                    "公司": job.get("brandName"),
                    "薪资": job.get("salaryDesc"),
                    "地区": job.get("cityName"),
                    "经验": job.get("jobExperience"),
                    "学历": job.get("jobDegree"),
                    "公司规模": job.get("brandScaleName"),
                    "行业": job.get("brandIndustry"),
                    "福利标签": ",".join(job.get("welfareList", [])),
                    "技能标签": ",".join(job.get("skills", [])),
                    "职位描述": job_desc
                }
                self.data_list.append(item)
                time.sleep(random.uniform(1, 1.5))

    def get_job_detail(self, job_id):
        url = f"https://www.zhipin.com/job_detail/{job_id}.html"
        try:
            resp = requests.get(url, headers=self.detail_headers)
            if resp.status_code != 200:
                return ""
            soup = BeautifulSoup(resp.text, 'html.parser')
            desc_tag = soup.select_one('.job-sec-text')
            return desc_tag.text.strip() if desc_tag else ""
        except Exception as e:
            print(f"详情页获取失败: {e}")
            return ""

    def save_excel(self):
        filename = f"python岗位数据_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df = pd.DataFrame(self.data_list)
        df.to_excel(filename, index=False)
        print(f"保存成功：{filename}，共 {len(df)} 条职位")

    def run(self):
        self.fetch_data()
        self.save_excel()

if __name__ == "__main__":
    spider = BossSpiderAPI()
    spider.run()



抓取第 1 页...
没有更多数据了
保存成功：python岗位数据_20251112_220055.xlsx，共 0 条职位


In [ ]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime
import re

class BossSpiderAPI:
    def __init__(self):
        """金融行业职位爬虫"""
        # 使用搜索接口
        self.base_url = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"
        
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
            "Referer": "https://www.zhipin.com/web/geek/job",
            "Accept": "application/json, text/plain, */*",
            "Content-Type": "application/x-www-form-urlencoded",
            "Cookie": "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; lastCity=100010000; __zp_stoken__=5a4ffw48LxIYLOBVRCWUOfcKFFkhJSHFuTUvCvGF6wrJgaMKwwrzCssK7wqzCtGzCoFbCgsK0worCucOFZsKZdMKvwq%2FCmFPCpcKTw7PCk8KMU8KSxJrDrcKExLHDuMKjwrzCpzA0FAwMDwPCh39%2FfHAPFwIBDQIKCgkVCwMDAAw5KMO8wqLCmzsyRz8hTVVVDVVcW0xYQRZRTUwzOAMBAww4Njw%2FM0TDhMKhwrPDrsK8wqHCsMOiwrvCqcKww6E%2FO0Q8wrPDui4wSwDCsBMICQB1D8OEw7EAw49kKMK0wq7CsiYgMjLCv8SyPzMeRTw7RTMwOzszLjAYw5RbIMK3wqHCuWkjPxwzMzM5PjszMzs8PS8zRzg1MzwvOAMDCwEJKkbCsMKDwrPDkTMz"
        }
        
        self.detail_headers = {
            "User-Agent": self.headers["User-Agent"],
            "Cookie": self.headers["Cookie"]
        }
        
        # 金融相关关键词（用于二次过滤）
        self.finance_pattern = re.compile(
            r"(金融|银行|券商|证券|基金|保险|期货|信托|投行|资管|理财|财富|"
            r"小贷|消金|保理|租赁|征信|清算|支付|互金|金融科技|FinTech)",
            re.IGNORECASE
        )
        
        self.data_list = []
        self.seen_ids = set()  # 去重

    def is_finance_related(self, job):
        """判断是否为金融相关职位"""
        text = " ".join([
            job.get("brandIndustry", "") or "",
            job.get("brandName", "") or "",
            job.get("jobName", "") or ""
        ])
        return bool(self.finance_pattern.search(text))

    def fetch_data(self, query="金融", max_pages=5):
        """
        抓取职位数据
        
        参数:
            query: 搜索关键词（可以是"金融"、"银行"等）
            max_pages: 最大抓取页数
        """
        for page in range(1, max_pages + 1):
            print(f"\n🔍 抓取关键词 [{query}] 第 {page} 页...")
            
            # 构建请求参数
            params = {
                "scene": "1",
                "query": query,
                "city": "100010000",  # 100010000=北京, 100000000=全国
                "page": page,
                "pageSize": 30
            }
            
            try:
                # 使用 POST 方法，参数放在 data 中
                resp = requests.post(
                    self.base_url, 
                    headers=self.headers, 
                    data=params,
                    timeout=15
                )
                resp.raise_for_status()
                
            except requests.RequestException as e:
                print(f"❌ 请求失败：{e}")
                continue

            # 解析响应
            try:
                result = resp.json()
                
                # 检查返回码
                code = result.get("code")
                msg = result.get("message") or result.get("msg")
                
                if code == 37:
                    print(f"⚠️ 触发风控 (code: 37)，请更新 Cookie！")
                    break
                elif code not in (0, "0", None):
                    print(f"⚠️ 接口异常 code: {code}, message: {msg}")
                    break
                    
            except Exception as e:
                print(f"❌ 解析响应失败: {e}")
                print(f"响应内容: {resp.text[:300]}")
                continue

            # 获取职位列表
            job_list = result.get("zpData", {}).get("jobList", [])
            
            if not job_list:
                print("⚠️ 本页无数据，结束抓取")
                break
            
            print(f"✅ 获取到 {len(job_list)} 个职位")
            
            # 处理每个职位
            finance_count = 0
            for job in job_list:
                job_id = job.get("encryptJobId")
                
                # 去重
                if not job_id or job_id in self.seen_ids:
                    continue
                
                # 金融行业过滤
                if not self.is_finance_related(job):
                    continue
                
                finance_count += 1
                brand_name = job.get("brandName")
                job_name = job.get("jobName")
                print(f"  💼 [{finance_count}] {job_name} - {brand_name}")
                
                # 获取职位详情
                job_desc = self.get_job_detail(job_id)
                
                # 构建数据项
                item = {
                    "职位": job_name,
                    "公司": brand_name,
                    "薪资": job.get("salaryDesc"),
                    "地区": job.get("cityName"),
                    "经验": job.get("jobExperience"),
                    "学历": job.get("jobDegree"),
                    "公司规模": job.get("brandScaleName"),
                    "行业": job.get("brandIndustry"),
                    "福利标签": ",".join(job.get("welfareList", []) or []),
                    "技能标签": ",".join(job.get("skills", []) or []),
                    "职位描述": job_desc
                }
                
                self.data_list.append(item)
                self.seen_ids.add(job_id)
                
                # 随机延时，避免被封
                time.sleep(random.uniform(1.5, 2.5))
            
            print(f"📊 本页符合条件: {finance_count} 个 | 累计: {len(self.data_list)} 个")
            
            # 每页之间延时
            time.sleep(random.uniform(2.0, 3.0))

    def get_job_detail(self, job_id):
        """获取职位详情描述"""
        url = f"https://www.zhipin.com/job_detail/{job_id}.html"
        try:
            resp = requests.get(url, headers=self.detail_headers, timeout=10)
            if resp.status_code != 200:
                return ""
            soup = BeautifulSoup(resp.text, 'html.parser')
            desc_tag = soup.select_one('.job-sec-text')
            return desc_tag.text.strip() if desc_tag else ""
        except Exception as e:
            print(f"  ⚠️ 详情页获取失败: {e}")
            return ""

    def save_excel(self):
        """保存为 Excel 文件"""
        if not self.data_list:
            print("\n❌ 没有数据可保存")
            return
        
        filename = f"金融行业岗位_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df = pd.DataFrame(self.data_list)
        df.to_excel(filename, index=False)
        print(f"\n✅ 保存成功：{filename}")
        print(f"   📦 共 {len(df)} 条职位")
        print(f"   📋 包含列: {', '.join(df.columns.tolist())}")

    def run(self, keywords=["金融"], max_pages_per_kw=3):
        """
        运行爬虫
        
        参数:
            keywords: 关键词列表，如 ["金融", "银行", "证券"]
            max_pages_per_kw: 每个关键词抓取的最大页数
        """
        print("🚀 开始抓取金融行业职位...")
        
        for keyword in keywords:
            print(f"\n{'='*60}")
            print(f"关键词: {keyword}")
            print('='*60)
            self.fetch_data(query=keyword, max_pages=max_pages_per_kw)
            
            # 关键词间延时
            if keyword != keywords[-1]:
                time.sleep(random.uniform(3.0, 5.0))
        
        self.save_excel()

if __name__ == "__main__":
    spider = BossSpiderAPI()
    
    # 可以搜索多个关键词
    keywords = ["金融", "银行", "证券", "基金", "保险"]
    
    # 每个关键词抓取 3 页（建议首次测试用 1-2 页）
    spider.run(keywords=keywords[:2], max_pages_per_kw=2)  # 先用前2个关键词测试

🚀 开始抓取金融行业职位...

关键词: 金融

🔍 抓取关键词 [金融] 第 1 页...
✅ 获取到 30 个职位
  💼 [1] 实习生（25届+金融学优先） - 江苏天鼎
  💼 [2] 学科编辑（金融学、经济学方向） - 智慧树
  💼 [3] 诚聘金融业储备管理+ 快速晋升+高薪2w+！ - 小企助力
  💼 [4] 案前外联调解员-金融业 - 邯郸律数
  💼 [5] 金融类  数据标注 - 四川知澳数据科技
  💼 [6] 金融类储备干部（涉外方向） - 华友钴业
  💼 [7] 金融业规划顾问+新人1W+入职有五险一金双休 - 玖橡咨询
  💼 [8] 网格员＋五险一金＋金融业＋周末双休 - 葫芦岛市天润科技
  💼 [9] 金融类标注-上市外企-六险一金 - 澳鹏科技
  💼 [10] 金融业合伙人 - 湖北晟邦信用管理
  💼 [11] 金融类项目助理 - 中财集团
  💼 [12] 金融业标注/双休五险一金 - 万声通讯实业有限公司
  💼 [13] 短视频编导（金融业） - 哆啦搜保
  💼 [14] 财经金融类记者 - 沈阳橙果文化传媒
  💼 [15] 秘书金融类 - 铭鼎信息科技有限公司


KeyboardInterrupt: 

In [3]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime
import re

class BossSpiderAPI:
    def __init__(self):
        """金融行业职位爬虫"""
        self.base_url = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"
        
        # ！！！注意：这里的 Cookie 必须是你刚从浏览器获取的最新值！！！
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
            "Referer": "https://www.zhipin.com/web/geek/job",
            "Accept": "application/json, text/plain, */*",
            "Content-Type": "application/x-www-form-urlencoded",
            # ！！！
            # ！！！把这里替换成你刚F12获取的最新Cookie
            # ！！！
            "Cookie": "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; lastCity=100010000; __zp_stoken__=5a4ffw47DpG0IPA1QFGYKDlNoSFVQQFloZE1LUm9USVjCsGTCksKrZV7CpsKwTcOEwqXCq8KuY0lAwr7DhsKiUMKqUsSFwqPCjsK9wpHEnsO1woXErMO8wqbDhMKRPTcAFA0CAHPCh35xcwsPAwwOFgILFBYPCwINDz0gw73Cr8KYPzpGMiJJTVQAVlhTTWVCAllMQTA8CwAODzwuPTIwMMK8wqDCvsOtwrjCqcKxw6%2FCuMKtwrjDoDI4METCssSHLURDAcK9EAwBAWgMwrDDuQHDgmcswpzCrkg0PjrCvcSzOz8mRz0%2FOTsyOj8%2FNjIMw4BnKMKlwqHCsMKbKz0dRz87Oz8%2FPzs5PTkjOzp2IT9ELTkXDwMDCC46wrjCgcKyw6U%2FOw%3D%3D" 
        }
        
        self.detail_headers = {
            "User-Agent": self.headers["User-Agent"],
            "Cookie": self.headers["Cookie"]
        }
        
        self.finance_pattern = re.compile(
            r"(金融|银行|券商|证券|基金|保险|期货|信托|投行|资管|理财|财富|"
            r"小贷|消金|保理|租赁|征信|清算|支付|互金|金融科技|FinTech)",
            re.IGNORECASE
        )
        
        self.data_list = []
        self.seen_ids = set()

    def is_finance_related(self, job):
        """判断是否为金融相关职位"""
        text = " ".join([
            job.get("brandIndustry", "") or "",
            job.get("brandName", "") or "",
            job.get("jobName", "") or ""
        ])
        return bool(self.finance_pattern.search(text))

    def fetch_data(self, query="金融", max_pages=5):
        """
        (修改) 抓取职位数据 - 仅抓取列表
        """
        for page in range(1, max_pages + 1):
            print(f"\n🔍 抓取关键词 [{query}] 列表第 {page} 页...")
            
            params = {
                "scene": "1",
                "query": query,
                "city": "100010000",
                "page": page,
                "pageSize": 30
            }
            
            try:
                resp = requests.post(
                    self.base_url, 
                    headers=self.headers, 
                    data=params,
                    timeout=15
                )
                resp.raise_for_status()
            except requests.RequestException as e:
                print(f"❌ 请求失败：{e}")
                continue

            try:
                result = resp.json()
                code = result.get("code")
                msg = result.get("message") or result.get("msg")
                
                if code == 37:
                    print(f"⚠️ 触发风控 (code: 37)，请更新 Cookie！")
                    break # 停止当前关键词的抓取
                elif code not in (0, "0", None):
                    print(f"⚠️ 接口异常 code: {code}, message: {msg}")
                    break
                    
            except Exception as e:
                print(f"❌ 解析响应失败: {e}")
                print(f"响应内容: {resp.text[:300]}")
                continue

            job_list = result.get("zpData", {}).get("jobList", [])
            
            if not job_list:
                print("⚠️ 本页无数据，结束抓取")
                break
            
            print(f"✅ 获取到 {len(job_list)} 个职位")
            
            finance_count = 0
            for job in job_list:
                job_id = job.get("encryptJobId")
                
                if not job_id or job_id in self.seen_ids:
                    continue
                
                if not self.is_finance_related(job):
                    continue
                
                finance_count += 1
                
                # *** (核心修改) ***
                # 先不获取详情，只保存列表信息
                item = {
                    "职位": job.get("jobName"),
                    "公司": job.get("brandName"),
                    "薪资": job.get("salaryDesc"),
                    "地区": job.get("cityName"),
                    "经验": job.get("jobExperience"),
                    "学历": job.get("jobDegree"),
                    "公司规模": job.get("brandScaleName"),
                    "行业": job.get("brandIndustry"),
                    "福利标签": ",".join(job.get("welfareList", []) or []),
                    "技能标签": ",".join(job.get("skills", []) or []),
                    "职位描述": "",  # 暂时为空
                    "job_id": job_id # 存储job_id，供后续抓取详情
                }
                
                self.data_list.append(item)
                self.seen_ids.add(job_id)
            
            print(f"📊 本页符合条件: {finance_count} 个 | 累计: {len(self.data_list)} 个")
            
            # 列表页之间的延时
            time.sleep(random.uniform(2.0, 3.0))

    def get_job_detail(self, job_id):
        """获取职位详情描述"""
        url = f"https://www.zhipin.com/job_detail/{job_id}.html"
        try:
            resp = requests.get(url, headers=self.detail_headers, timeout=10)
            if resp.status_code != 200:
                print(f"  ⚠️ 详情页状态码: {resp.status_code}")
                # 遇到403或404等也可能意味着被风控
                return ""
            soup = BeautifulSoup(resp.text, 'html.parser')
            desc_tag = soup.select_one('.job-sec-text')
            return desc_tag.text.strip() if desc_tag else ""
        except Exception as e:
            print(f"  ⚠️ 详情页获取失败: {e}")
            return ""

    # *** (新增方法) ***
    def fetch_all_details(self):
        """
        在列表抓取完毕后，统一抓取所有详情
        """
        print(f"\n{'='*60}")
        print(f"🚀 开始抓取 {len(self.data_list)} 个职位的详细描述...")
        print("   (此过程较慢，请耐心等待)")
        print(f"{'='*60}")
        
        for i, item in enumerate(self.data_list):
            job_id = item["job_id"]
            
            print(f"  ({i+1}/{len(self.data_list)}) 抓取: {item['职位']} - {item['公司']}")
            
            desc = self.get_job_detail(job_id)
            item["职位描述"] = desc
            
            # *** (核心修改) ***
            # 拉长详情页的抓取间隔，模拟人类点击速度
            sleep_time = random.uniform(3.0, 5.0) 
            time.sleep(sleep_time)
            
        print("\n✅ 所有职位描述抓取完毕。")


    def save_excel(self):
        """保存为 Excel 文件"""
        if not self.data_list:
            print("\n❌ 没有数据可保存")
            return
        
        # (修改) 在保存前，移除临时的 'job_id' 列
        df = pd.DataFrame(self.data_list)
        if 'job_id' in df.columns:
            df = df.drop(columns=['job_id'])
            
        filename = f"金融行业岗位_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(filename, index=False)
        print(f"\n✅ 保存成功：{filename}")
        print(f"    📦 共 {len(df)} 条职位")
        print(f"    📋 包含列: {', '.join(df.columns.tolist())}")

    def run(self, keywords=["金融"], max_pages_per_kw=3):
        """
        运行爬虫
        """
        print("🚀 (阶段1) 开始抓取职位列表...")
        
        for keyword in keywords:
            print(f"\n{'='*60}")
            print(f"关键词: {keyword}")
            print('='*60)
            self.fetch_data(query=keyword, max_pages=max_pages_per_kw)
            
            if keyword != keywords[-1]:
                time.sleep(random.uniform(3.0, 5.0))
        
        print("\n👍 (阶段1) 职位列表抓取完毕。")
        
        # (核心修改) 在列表抓取完后，再统一抓取详情
        if self.data_list:
            self.fetch_all_details()
        
        # (核心修改) 最后保存
        self.save_excel()

if __name__ == "__main__":
    spider = BossSpiderAPI()
    
    keywords = ["金融", "银行", "证券", "基金", "保险"]
    
    # 假设你每个关键词要10页
    # spider.run(keywords=keywords, max_pages_per_kw=10) 
    
    # 测试时，先用少量数据
    spider.run(keywords=keywords[:2], max_pages_per_kw=2)

🚀 (阶段1) 开始抓取职位列表...

关键词: 金融

🔍 抓取关键词 [金融] 列表第 1 页...
✅ 获取到 30 个职位
📊 本页符合条件: 30 个 | 累计: 30 个

🔍 抓取关键词 [金融] 列表第 2 页...
✅ 获取到 30 个职位
📊 本页符合条件: 11 个 | 累计: 41 个

关键词: 银行

🔍 抓取关键词 [银行] 列表第 1 页...
✅ 获取到 30 个职位
📊 本页符合条件: 27 个 | 累计: 68 个

🔍 抓取关键词 [银行] 列表第 2 页...
⚠️ 触发风控 (code: 37)，请更新 Cookie！

👍 (阶段1) 职位列表抓取完毕。

🚀 开始抓取 68 个职位的详细描述...
   (此过程较慢，请耐心等待)
  (1/68) 抓取: 实习生（25届+金融学优先） - 江苏天鼎
  (2/68) 抓取: 学科编辑（金融学、经济学方向） - 智慧树
  (3/68) 抓取: 诚聘金融业储备管理+ 快速晋升+高薪2w+！ - 小企助力
  (4/68) 抓取: 案前外联调解员-金融业 - 邯郸律数
  (5/68) 抓取: 金融类  数据标注 - 四川知澳数据科技
  (6/68) 抓取: 金融类储备干部（涉外方向） - 华友钴业
  (7/68) 抓取: 金融业规划顾问+新人1W+入职有五险一金双休 - 玖橡咨询
  (8/68) 抓取: 网格员＋五险一金＋金融业＋周末双休 - 葫芦岛市天润科技
  (9/68) 抓取: 金融类标注-上市外企-六险一金 - 澳鹏科技
  (10/68) 抓取: 金融业合伙人 - 湖北晟邦信用管理
  (11/68) 抓取: 金融类项目助理 - 中财集团
  (12/68) 抓取: 金融业标注/双休五险一金 - 万声通讯实业有限公司
  (13/68) 抓取: 短视频编导（金融业） - 哆啦搜保
  (14/68) 抓取: 财经金融类记者 - 沈阳橙果文化传媒
  (15/68) 抓取: 秘书金融类 - 铭鼎信息科技有限公司
  (16/68) 抓取: 高薪招聘英文金融类讲解员 - 泰盈科技集团
  (17/68) 抓取: 泛商业、金融类新媒体运营 - 山东世裕
  (18/68) 抓取: 客户邀约专员（金融类） - 山东沁远企业咨询服务
  (19/68) 抓取:

In [ ]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime
import re

class BossSpiderAPI:
    def __init__(self):
        """金融行业职位爬虫 - 优化反爬版"""
        self.base_url = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"
        self.search_page = "https://www.zhipin.com/web/geek/job"
        
        # ========== 方法2: Cookie 轮换池（准备2-3个Cookie） ==========
        self.cookies = [
            # Cookie 1: 从浏览器复制第一个账号的完整Cookie
            "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; lastCity=100010000; __zp_stoken__=5a4ffw47DvcORCzMIUghlFQtRdEtKZUJla1tISV5sS0xawrxnwp3CrmVlTE7CvsK6wqJewq5aw7NLwqlVwqtRwpbCqMO8wpHCp8K9wpLEkcOgwofEvsOKVMKxwqQxNA8BDw4DfHJ8fXAUCgEADQkXCQgVAA4AAQwyNcO%2FwqPCmzA%2FRD4hVkhWDFVnZk9ZQQ1cTk0zMw4CAgwzKz8%2BMz%2FCucKiwrLDrsOHwqzCs8OjwrvCosK9w6I%2BOz8xwrDDuy47VgPCsRMDFAN0D8K%2Fw7wDw45kI8Krw7%2FCsHQrPzHCvsSyRD4dRDwwODAxOzA%2BLTFFw49mI8K0w7PCscKMID4cOD4wOD4wPjA6PEYiMDkrLj4%2FLjgIDggACSE7wrPCgsKzw5o%2BMA%3D%3D",
            
            # Cookie 2: 从浏览器复制第二个账号的完整Cookie（替换为你的第二个Cookie）
            "__zp_seo_uuid__=1190494f-f11f-4a57-b5e8-f706bee381e9; __l=r=https%3A%2F%2Fwww.google.com%2F&l=%2F&s=1; __g=-; lastCity=100010000; ab_guid=8dd2897e-0c3e-4151-9bb1-fa78a70c1d18; __zp_stoken__=5a4ffw47Ct8KWCTIIUAtnFHDChBRKQ1VwbE9BwrFgeMKwanXCscK%2BwrDCscKhwqhRwo53wptVwqpOwqNYwr7CqsK6wr3CuMKTw6zCq8KawpnCncK%2BwpDEkMOgwoXEssOsw4TCscKQMjYOAQ0NAX1yfn5yFQoDAw8IFwsLFwEOAgIOMzXDvcKgwpkxP0Y9I1dIVA9XZmZNWkMMXExOMTIOAAEOMis9PTE%2BwrnCoMKxw6zDhsKswrHDoMK5wqPCvcOgPTk%2BMcKyw7gsOlYBwrIRAhQBdw3CvsO8AcONZiLCqMKpYzQwP8K9xLw6MRNHMj5HPjJFPjEjMgzDgVktwrTCqVohMRI6PjFHMDI%2BMUUyRCIxRiwsPj4hRgoOCQ8XIzvCssKNwr3DmD4x; __c=1762957164; __a=91473911.1762957164..1762957164.5.1.5.5"
        ]
        self.current_cookie_index = 0
        
        # ========== 方法4: 增强请求头池（更多浏览器特征） ==========
        self.user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
        ]
        
        # 金融相关关键词（用于二次过滤）
        self.finance_pattern = re.compile(
            r"(金融|银行|券商|证券|基金|保险|期货|信托|投行|资管|理财|财富|"
            r"小贷|消金|保理|租赁|征信|清算|支付|互金|金融科技|FinTech)",
            re.IGNORECASE
        )
        
        self.data_list = []
        self.seen_ids = set()  # 去重
        self.request_count = 0  # 请求计数

    def rotate_cookie(self):
        """Cookie 轮换策略"""
        self.current_cookie_index = (self.current_cookie_index + 1) % len(self.cookies)
        print(f"🔄 切换到 Cookie #{self.current_cookie_index + 1}")

    def get_random_headers(self, include_cookie=True):
        """生成随机请求头（方法4增强版）"""
        headers = {
            "User-Agent": random.choice(self.user_agents),
            "Referer": "https://www.zhipin.com/web/geek/job",
            "Accept": "application/json, text/plain, */*",
            "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
            "Accept-Encoding": "gzip, deflate, br",
            "Connection": "keep-alive",
            "Content-Type": "application/x-www-form-urlencoded",
            "Origin": "https://www.zhipin.com",
            "Sec-Fetch-Dest": "empty",
            "Sec-Fetch-Mode": "cors",
            "Sec-Fetch-Site": "same-origin",
            "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"Windows"',
            "Upgrade-Insecure-Requests": "1"
        }
        
        if include_cookie:
            headers["Cookie"] = self.cookies[self.current_cookie_index]
        
        return headers

    def is_finance_related(self, job):
        """判断是否为金融相关职位"""
        text = " ".join([
            job.get("brandIndustry", "") or "",
            job.get("brandName", "") or "",
            job.get("jobName", "") or ""
        ])
        return bool(self.finance_pattern.search(text))

    def warmup_session(self, session):
        """预热会话：模拟真实用户行为"""
        print("🔥 正在预热连接...")
        try:
            headers = self.get_random_headers()
            _ = session.get(
                self.search_page, 
                headers=headers,
                params={"query": "金融", "city": "100010000"},
                timeout=15
            )
            # 方法1: 增加预热延时到 5-8 秒
            sleep_time = random.uniform(5.0, 8.0)
            print(f"   💤 预热延时 {sleep_time:.1f} 秒...")
            time.sleep(sleep_time)
        except Exception as e:
            print(f"   ⚠️ 预热失败（可继续）: {e}")

    def fetch_data(self, query="金融", max_pages=5):
        """
        抓取职位数据（方法1+2+3优化版）
        
        参数:
            query: 搜索关键词
            max_pages: 最大抓取页数（建议不超过3）
        """
        session = requests.Session()
        
        # 预热会话
        self.warmup_session(session)
        
        for page in range(1, max_pages + 1):
            print(f"\n{'='*70}")
            print(f"🔍 抓取关键词 [{query}] 第 {page} 页...")
            print(f"📊 当前统计: 已采集 {len(self.data_list)} 条 | 请求 {self.request_count} 次")
            print('='*70)
            
            # ========== 方法2: 每2页轮换一次Cookie ==========
            if page > 1 and page % 2 == 0:
                self.rotate_cookie()
                # Cookie切换后额外延时
                time.sleep(random.uniform(3.0, 5.0))
            
            # ========== 方法1: 页面间大幅增加延时（10-20秒） ==========
            if page > 1:
                sleep_time = random.uniform(10.0, 20.0)
                print(f"💤 页面间延时 {sleep_time:.1f} 秒...")
                time.sleep(sleep_time)
            
            # 构建请求参数
            params = {
                "scene": "1",
                "query": query,
                "city": "100010000",  # 100010000=北京, 100000000=全国
                "page": page,
                "pageSize": 20  # 方法3: 降低每页数量到20
            }
            
            try:
                # 方法4: 每次请求更换请求头
                headers = self.get_random_headers()
                
                resp = session.post(
                    self.base_url, 
                    headers=headers, 
                    data=params,
                    timeout=15
                )
                resp.raise_for_status()
                self.request_count += 1
                
            except requests.RequestException as e:
                print(f"❌ 请求失败：{e}")
                # 失败后尝试切换Cookie
                self.rotate_cookie()
                time.sleep(random.uniform(5.0, 10.0))
                continue

            # 解析响应
            try:
                result = resp.json()
                code = result.get("code")
                msg = result.get("message") or result.get("msg")
                
                if code == 37:
                    print(f"⚠️ 触发风控 (code: 37): {msg}")
                    print("\n💡 建议操作：")
                    print("   1. 当前数据已保存")
                    print("   2. 尝试切换到下一个Cookie继续...")
                    self.rotate_cookie()
                    time.sleep(60)  # 等待1分钟
                    continue
                    
                elif code not in (0, "0", None):
                    print(f"⚠️ 接口异常 code: {code}, message: {msg}")
                    break
                    
            except Exception as e:
                print(f"❌ 解析响应失败: {e}")
                print(f"响应内容: {resp.text[:300]}")
                continue

            # 获取职位列表
            job_list = result.get("zpData", {}).get("jobList", [])
            
            if not job_list:
                print("⚠️ 本页无数据")
                break
            
            print(f"✅ 本页获取到 {len(job_list)} 个职位")
            
            # 处理每个职位
            finance_count = 0
            for idx, job in enumerate(job_list, 1):
                job_id = job.get("encryptJobId")
                
                # 去重
                if not job_id or job_id in self.seen_ids:
                    continue
                
                # 金融行业过滤
                if not self.is_finance_related(job):
                    continue
                
                finance_count += 1
                brand_name = job.get("brandName")
                job_name = job.get("jobName")
                print(f"  💼 [{finance_count}] {job_name} - {brand_name}")
                
                # 获取职位详情
                job_desc = self.get_job_detail(session, job_id)
                
                # 构建数据项
                item = {
                    "职位": job_name,
                    "公司": brand_name,
                    "薪资": job.get("salaryDesc"),
                    "地区": job.get("cityName"),
                    "经验": job.get("jobExperience"),
                    "学历": job.get("jobDegree"),
                    "公司规模": job.get("brandScaleName"),
                    "行业": job.get("brandIndustry"),
                    "福利标签": ",".join(job.get("welfareList", []) or []),
                    "技能标签": ",".join(job.get("skills", []) or []),
                    "职位描述": job_desc
                }
                
                self.data_list.append(item)
                self.seen_ids.add(job_id)
                
                # ========== 方法1: 详情页延时增加到 8-15 秒 ==========
                sleep_time = random.uniform(8.0, 15.0)
                print(f"       ⏳ 延时 {sleep_time:.1f} 秒...")
                time.sleep(sleep_time)
            
            print(f"📊 本页符合条件: {finance_count} 个 | 累计: {len(self.data_list)} 个")

    def get_job_detail(self, session, job_id):
        """获取职位详情描述"""
        if not job_id:
            return ""
        
        url = f"https://www.zhipin.com/job_detail/{job_id}.html"
        try:
            # 使用随机请求头
            headers = self.get_random_headers()
            resp = session.get(url, headers=headers, timeout=15)
            
            if resp.status_code != 200:
                return ""
            
            soup = BeautifulSoup(resp.text, 'html.parser')
            desc_tag = soup.select_one('.job-sec-text')
            return desc_tag.text.strip() if desc_tag else ""
            
        except Exception as e:
            print(f"       ⚠️ 详情页获取失败: {e}")
            return ""

    def save_excel(self):
        """保存为 Excel 文件"""
        if not self.data_list:
            print("\n❌ 没有数据可保存")
            return
        
        filename = f"金融行业岗位_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df = pd.DataFrame(self.data_list)
        df.to_excel(filename, index=False)
        
        print(f"\n{'='*70}")
        print(f"✅ 保存成功：{filename}")
        print(f"   📦 共 {len(df)} 条职位（去重后）")
        print(f"   🔢 总请求数: {self.request_count} 次")
        print(f"   📋 包含列: {', '.join(df.columns.tolist())}")
        print('='*70)

    def run(self, keywords=["金融"], max_pages_per_kw=3):
        """
        运行爬虫（方法3: 分批采集）
        
        参数:
            keywords: 关键词列表
            max_pages_per_kw: 每个关键词抓取的最大页数（建议2-3页）
        """
        print("🚀 开始抓取金融行业职位...")
        print(f"📝 配置: {len(self.cookies)} 个Cookie, {len(keywords)} 个关键词")
        print(f"⚙️  策略: 每页延时10-20秒, 详情延时8-15秒, 每2页切换Cookie")
        
        for keyword in keywords:
            print(f"\n{'#'*70}")
            print(f"# 关键词: {keyword}")
            print('#'*70)
            
            self.fetch_data(query=keyword, max_pages=max_pages_per_kw)
            
            # ========== 方法3: 关键词间大幅延时（30-60秒） ==========
            if keyword != keywords[-1]:
                sleep_time = random.uniform(30.0, 60.0)
                print(f"\n⏸️  关键词切换，休息 {sleep_time:.1f} 秒...")
                time.sleep(sleep_time)
        
        self.save_excel()

if __name__ == "__main__":
    spider = BossSpiderAPI()
    
    # ========== 方法3: 分批采集策略 ==========
    # 建议: 首次只爬1个关键词2页，测试通过后再增加
    keywords = ["金融"]  # 可扩展: ["金融", "银行", "证券"]
    
    spider.run(keywords=keywords, max_pages_per_kw=2)  # 先测试2页
    
    # 如果需要更多数据，间隔30-60分钟后再运行下一批
    print("\n💡 提示: 如需更多数据，请等待30-60分钟后再次运行")

🚀 开始抓取金融行业职位...
📝 配置: 2 个Cookie, 1 个关键词
⚙️  策略: 每页延时10-20秒, 详情延时8-15秒, 每2页切换Cookie

######################################################################
# 关键词: 金融
######################################################################
🔥 正在预热连接...
   💤 预热延时 5.4 秒...

🔍 抓取关键词 [金融] 第 1 页...
📊 当前统计: 已采集 0 条 | 请求 0 次
⚠️ 触发风控 (code: 37): 您的访问行为异常.

💡 建议操作：
   1. 当前数据已保存
   2. 尝试切换到下一个Cookie继续...
🔄 切换到 Cookie #2

🔍 抓取关键词 [金融] 第 2 页...
📊 当前统计: 已采集 0 条 | 请求 1 次
🔄 切换到 Cookie #1
💤 页面间延时 15.9 秒...
⚠️ 触发风控 (code: 37): 您的访问行为异常.

💡 建议操作：
   1. 当前数据已保存
   2. 尝试切换到下一个Cookie继续...
🔄 切换到 Cookie #2
